## Notebook demo: RTMD database

This notebook now includes a short Python snippet to connect to the local RTMD PostgreSQL instance and run a quick `COUNT(*)` test against `social_posts`.

For a more complete demo and partition management, see `python/connect_and_demo.py` and `scripts/create_month_partitions.ps1`.

Steps:
1. Ensure DB is running (`docker compose up -d`).
2. Apply migrations (`scripts\apply_migrations.ps1`).
3. Run the code cell below to confirm connection and counts.


In [ ]:
# Demo: Connect to RTMD DB (Python)

# Ensure you have installed dependencies: `pip install -r requirements.txt`
# Set environment variables (or copy `.env.example` to `.env` and load it).

import os
import psycopg2
from psycopg2.extras import RealDictCursor

DB_HOST = os.getenv('DB_HOST','localhost')
DB_PORT = int(os.getenv('DB_PORT',5432))
DB_NAME = os.getenv('DB_NAME','rtmd')
DB_USER = os.getenv('DB_USER','rtmd_user')
DB_PASS = os.getenv('DB_PASS','change_me')

print('DB connection settings:', DB_HOST, DB_PORT, DB_NAME, DB_USER)

try:
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        cur.execute("SELECT count(*) AS total FROM social_posts;")
        print('social_posts count:', cur.fetchone()['total'])
    conn.close()
except Exception as e:
    print('Connection failed:', e)


In [ ]:
import sqlite3

# This will work without any installation
conn = sqlite3.connect('my_database.db')
print("Connection successful!")


print(sqlite3.version)

Connection successful!
2.6.0


/tmp/ipython-input-1522937468.py:8: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)


In [ ]:
import pandas as pd

tables = [
    'User', 'Location', 'Disaster', 'Disaster_Location',
    'Credibility', 'Stream', 'Post', 'Model_Result'
]

for table in tables:
    try:
        df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 3", conn)
        print(f"\n{table} — first few rows:")
        print(df)
        print(f"Total rows: {len(pd.read_sql_query(f'SELECT * FROM {table}', conn))}")
    except Exception as e:
        print(f"Problem with {table}: {e}")

Problem with User: Execution failed on sql 'SELECT * FROM User LIMIT 3': no such table: User
Problem with Location: Execution failed on sql 'SELECT * FROM Location LIMIT 3': no such table: Location
Problem with Disaster: Execution failed on sql 'SELECT * FROM Disaster LIMIT 3': no such table: Disaster
Problem with Disaster_Location: Execution failed on sql 'SELECT * FROM Disaster_Location LIMIT 3': no such table: Disaster_Location
Problem with Credibility: Execution failed on sql 'SELECT * FROM Credibility LIMIT 3': no such table: Credibility
Problem with Stream: Execution failed on sql 'SELECT * FROM Stream LIMIT 3': no such table: Stream
Problem with Post: Execution failed on sql 'SELECT * FROM Post LIMIT 3': no such table: Post
Problem with Model_Result: Execution failed on sql 'SELECT * FROM Model_Result LIMIT 3': no such table: Model_Result


In [ ]:
# Step 1: Mount Drive (run this first every time after restart)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Step 2: Connect to the SAME persistent file
import sqlite3
DB_PATH = '/content/drive/MyDrive/disaster_monitoring.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

print("Connected to:", DB_PATH)

# Very important in SQLite - enable foreign keys
cursor.execute("PRAGMA foreign_keys = ON;")

# Step 3: Create ALL tables (IF NOT EXISTS so it's safe to re-run)
cursor.executescript('''
CREATE TABLE IF NOT EXISTS User (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    password_hash TEXT NOT NULL,
    role TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS Location (
    location_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    district TEXT,
    province TEXT
);

CREATE TABLE IF NOT EXISTS Disaster (
    disaster_id INTEGER PRIMARY KEY AUTOINCREMENT,
    disaster_type TEXT NOT NULL,
    severity TEXT,
    confidence_score REAL DEFAULT 0.0,
    date_time TEXT NOT NULL,
    status TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS Disaster_Location (
    disaster_id INTEGER NOT NULL,
    location_id INTEGER NOT NULL,
    PRIMARY KEY (disaster_id, location_id),
    FOREIGN KEY (disaster_id) REFERENCES Disaster(disaster_id) ON DELETE CASCADE,
    FOREIGN KEY (location_id) REFERENCES Location(location_id) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Credibility (
    credibility_id INTEGER PRIMARY KEY AUTOINCREMENT,
    score REAL NOT NULL DEFAULT 0.5,
    verification_status TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS Stream (
    stream_id INTEGER PRIMARY KEY AUTOINCREMENT,
    source_platform TEXT NOT NULL,
    last_update_timestamp TEXT NOT NULL DEFAULT (datetime('now'))
);

CREATE TABLE IF NOT EXISTS Post (
    post_id INTEGER PRIMARY KEY,
    post_text TEXT,
    post_image TEXT,
    language TEXT,
    platform TEXT NOT NULL,
    timestamp TEXT NOT NULL,
    credibility_id INTEGER,
    stream_id INTEGER,
    disaster_id INTEGER,
    FOREIGN KEY (credibility_id) REFERENCES Credibility(credibility_id) ON DELETE SET NULL,
    FOREIGN KEY (stream_id)      REFERENCES Stream(stream_id)          ON DELETE SET NULL,
    FOREIGN KEY (disaster_id)    REFERENCES Disaster(disaster_id)      ON DELETE SET NULL
);

CREATE TABLE IF NOT EXISTS Model_Result (
    result_id INTEGER PRIMARY KEY AUTOINCREMENT,
    post_id INTEGER NOT NULL,
    accuracy REAL,
    disaster_label TEXT,
    model_type TEXT NOT NULL,
    confidence_score REAL NOT NULL DEFAULT 0.0,
    FOREIGN KEY (post_id) REFERENCES Post(post_id) ON DELETE CASCADE
);
''')

conn.commit()
print("All tables are now created (or already existed).")

Mounted at /content/drive
Connected to: /content/drive/MyDrive/disaster_monitoring.db
All tables are now created (or already existed).


In [ ]:
import pandas as pd

# Quick test: see if tables exist now
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Existing tables:", [t[0] for t in tables])

# Try one table
print("\nUsers table content:")
print(pd.read_sql_query("SELECT * FROM User LIMIT 3", conn))

Existing tables: ['User', 'sqlite_sequence', 'Location', 'Disaster', 'Disaster_Location', 'Credibility', 'Stream', 'Post', 'Model_Result']

Users table content:
Empty DataFrame
Columns: [user_id, name, email, password_hash, role]
Index: []


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import sqlite3

# Re-establish connection and cursor for robustness against kernel restarts
DB_PATH = '/content/drive/MyDrive/disaster_monitoring.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Quick test: see if tables exist now
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Existing tables:", [t[0] for t in tables])

# Try one table
print("\nUsers table content:")
print(pd.read_sql_query("SELECT * FROM User LIMIT 3", conn))

Mounted at /content/drive
Existing tables: ['User', 'sqlite_sequence', 'Location', 'Disaster', 'Disaster_Location', 'Credibility', 'Stream', 'Post', 'Model_Result']

Users table content:
Empty DataFrame
Columns: [user_id, name, email, password_hash, role]
Index: []


In [ ]:
cursor.execute("""
CREATE VIEW IF NOT EXISTS v_active_disasters AS
SELECT
    d.disaster_id,
    d.disaster_type,
    d.severity,
    d.confidence_score,
    d.date_time,
    d.status,
    GROUP_CONCAT(l.name || ' (' || l.province || ')') AS affected_locations
FROM Disaster d
LEFT JOIN Disaster_Location dl ON d.disaster_id = dl.disaster_id
LEFT JOIN Location l ON dl.location_id = l.location_id
WHERE d.status IN ('Active', 'Monitoring')
GROUP BY d.disaster_id
ORDER BY d.date_time DESC;
""")

cursor.execute("""
CREATE VIEW IF NOT EXISTS v_high_confidence_posts AS
SELECT
    p.post_id,
    p.post_text,
    p.platform,
    p.timestamp,
    p.language,
    c.score AS credibility_score,
    c.verification_status,
    mr.disaster_label,
    mr.confidence_score AS model_confidence
FROM Post p
LEFT JOIN Credibility c ON p.credibility_id = c.credibility_id
LEFT JOIN Model_Result mr ON p.post_id = mr.post_id
WHERE mr.confidence_score > 0.75
ORDER BY mr.confidence_score DESC;
""")

cursor.execute("""
CREATE VIEW IF NOT EXISTS v_posts_per_disaster AS
SELECT
    d.disaster_id,
    d.disaster_type,
    d.severity,
    COUNT(p.post_id) AS post_count,
    COUNT(CASE WHEN p.language = 'si' THEN 1 END) AS sinhala_posts,
    COUNT(CASE WHEN p.language = 'ta' THEN 1 END) AS tamil_posts,
    AVG(mr.confidence_score) AS avg_model_confidence
FROM Disaster d
LEFT JOIN Post p ON d.disaster_id = p.disaster_id
LEFT JOIN Model_Result mr ON p.post_id = mr.post_id
GROUP BY d.disaster_id
ORDER BY post_count DESC;
""")

conn.commit()
print("3 main views created.")

# Quick check
print("\nActive disasters view:")
print(pd.read_sql_query("SELECT * FROM v_active_disasters", conn))

3 main views created.

Active disasters view:
Empty DataFrame
Columns: [disaster_id, disaster_type, severity, confidence_score, date_time, status, affected_locations]
Index: []


In [ ]:
queries = [
    ("Active disasters with locations", "SELECT * FROM v_active_disasters"),
    ("High confidence detections", "SELECT * FROM v_high_confidence_posts LIMIT 5"),
    ("Posts count & language breakdown per disaster", "SELECT * FROM v_posts_per_disaster"),
    ("Most recent 5 posts overall", "SELECT post_id, platform, timestamp, post_text FROM Post ORDER BY timestamp DESC LIMIT 5"),
    ("Average model confidence per disaster type", """
        SELECT d.disaster_type, AVG(mr.confidence_score) AS avg_conf
        FROM Disaster d
        JOIN Post p ON d.disaster_id = p.disaster_id
        JOIN Model_Result mr ON p.post_id = mr.post_id
        GROUP BY d.disaster_type
        ORDER BY avg_conf DESC
    """),
    ("Posts from low-credibility sources", """
        SELECT p.platform, p.post_text, c.score, c.verification_status
        FROM Post p
        JOIN Credibility c ON p.credibility_id = c.credibility_id
        WHERE c.score < 0.5
    """)
]

for title, sql in queries:
    print(f"\n{title}:")
    try:
        df = pd.read_sql_query(sql, conn)
        print(df)
    except Exception as e:
        print(f"Error: {e}")


Active disasters with locations:
Empty DataFrame
Columns: [disaster_id, disaster_type, severity, confidence_score, date_time, status, affected_locations]
Index: []

High confidence detections:
Empty DataFrame
Columns: [post_id, post_text, platform, timestamp, language, credibility_score, verification_status, disaster_label, model_confidence]
Index: []

Posts count & language breakdown per disaster:
Empty DataFrame
Columns: [disaster_id, disaster_type, severity, post_count, sinhala_posts, tamil_posts, avg_model_confidence]
Index: []

Most recent 5 posts overall:
Empty DataFrame
Columns: [post_id, platform, timestamp, post_text]
Index: []

Average model confidence per disaster type:
Empty DataFrame
Columns: [disaster_type, avg_conf]
Index: []

Posts from low-credibility sources:
Empty DataFrame
Columns: [platform, post_text, score, verification_status]
Index: []


In [ ]:
# Should fail (duplicate email)
try:
    cursor.execute("INSERT INTO User (name, email, password_hash, role) VALUES ('Test Dup', 'admin@disaster.lk', 'xxx', 'test')")
    conn.commit()
    print("Duplicate email → unexpectedly succeeded")
except sqlite3.IntegrityError:
    print("Duplicate email → correctly blocked (UNIQUE constraint works)")

# Cascade test
cursor.execute("DELETE FROM Disaster WHERE disaster_id = 4")   # assuming you have disaster_id=4
conn.commit()
print("After deleting disaster 4 → check if related Disaster_Location rows are gone:")
print(pd.read_sql_query("SELECT * FROM Disaster_Location WHERE disaster_id = 4", conn))

Duplicate email → unexpectedly succeeded
After deleting disaster 4 → check if related Disaster_Location rows are gone:
Empty DataFrame
Columns: [disaster_id, location_id]
Index: []
